# Get started with Metrics Tracking and Monitoring

This notebook demonstrates how to use Evidently to:
* Generate a model performance report and calculate associated metrics.
* Log model metrics to MLFlow.
* Store the model in MLFlow as an artifact.
* Store the model performance report in MLFlow as an artifact.

In [1]:
%load_ext autoreload
%autoreload 2

import joblib
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
import pandas as pd
from pathlib import Path
from sklearn import ensemble, model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

from config import MLFLOW_TRACKING_URI

## Load Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [2]:
# Download original dataset with: python src/pipelines/load_data.py 

raw_data = pd.read_csv(f"../data/raw_data.csv")
raw_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Define column mapping

In [3]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]

In [4]:
sample_data = raw_data.set_index('dteday').loc['2011-01-01 00:00:00':'2011-01-28 23:00:00'].reset_index()

print(sample_data.shape)

(594, 17)


In [5]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    sample_data[numerical_features + categorical_features],
    sample_data[target],
    test_size=0.3
)

print(X_train.shape)
print(X_test.shape)

(415, 10)
(179, 10)


## Train a  Linear Regression Model

In [6]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train) 

model_lr_path = Path('../models/model_lr.joblib')
joblib.dump(model_lr, model_lr_path)

['../models/model_lr.joblib']

In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

preds_lr = model_lr.predict(X_test)

me = mean_squared_error(y_test, preds_lr)
mae = mean_absolute_error(y_test, preds_lr)

print(me, mae)

2531.610953803402 36.12775337079473


## Train a RandomForestRegressor Model

In [8]:
model_rf = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)
model_rf.fit(X_train, y_train) 

model_path = Path('../models/model_rf.joblib')
joblib.dump(model_rf, model_path)

['../models/model_rf.joblib']

In [9]:

preds_rf = model_rf.predict(X_test)

me = mean_squared_error(y_test, preds_rf)
mae = mean_absolute_error(y_test, preds_rf)

print(me, mae)

382.5649340782123 12.801787709497209


## Build the model validation report

In [10]:
report = X_test.copy()
report['target'] = y_test
report['prediction_lr'] = preds_lr
report['prediction_rf'] = preds_rf

report.head()

,temp,atemp,hum,windspeed,mnth,hr,weekday,season,holiday,workingday,target,prediction_lr,prediction_rf
478,0.04,0.0758,0.57,0.1045,1,0,0,1,0,0,22,-3.115272,21.02
88,0.20,0.1970,0.51,0.1940,1,21,3,1,0,1,43,82.108524,41.12
517,0.14,0.1212,0.50,0.2537,1,17,1,1,0,1,184,62.738368,131.18
360,0.18,0.1970,0.47,0.1343,1,12,1,1,1,0,80,42.289583,77.94
127,0.20,0.1970,0.40,0.2537,1,14,5,1,0,1,50,76.448178,63.72


# Model Registry

## Set up MLFlow

In [11]:
# Set up MLFlow 
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# Set up MLFlow Client
client = MlflowClient()
print(f"Client tracking uri: {client.tracking_uri}")


Client tracking uri: http://localhost:5000


## Registering a Model

- Docs on [mlflow.sklearn.log_model](https://www.mlflow.org/docs/latest/python_api/mlflow.sklearn.html?highlight=save_model#mlflow.sklearn.log_model)

### Log the `model_lr` model

In [12]:
with mlflow.start_run() as run: 
    
    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=model_lr,
        artifact_path="LinearRegression"
    )

### Log and Register the `model_rf` model

- Use `registered_model_name` to register a model automatically.
- If a registered model with the name doesn’t exist, the method registers a new model and creates `Version 1`.
- If a registered model with the name exists, the method creates a new model version.

INSTRUCTION: 
- Run the cell below 3 time to register 3 versions of the "bike-sharing-RandomForestRegressor" model

In [17]:
from mlflow.models import infer_signature

with mlflow.start_run() as run: 

    # Show newly created run metadata info
    print("Experiment id: {}".format(run.info.experiment_id))
    print("Run id: {}".format(run.info.run_id))
    print("Run name: {}".format(run.info.run_name))
    print('MLFlow tracking uri:', mlflow.get_tracking_uri())
    print('MLFlow artifact uri:', mlflow.get_artifact_uri())
    run_id = run.info.run_id

    # Infer the model signature
    signature = infer_signature(y_test, preds_rf)

    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=model_rf,
        artifact_path="RandomForest",
        signature=signature,
        registered_model_name="RandomForest",
    )

Experiment id: 0
Run id: 1c05fa31f83440829d746e7cd7d1adb6
Run name: resilient-snake-802
MLFlow tracking uri: http://localhost:5000
MLFlow artifact uri: ./mlruns/0/1c05fa31f83440829d746e7cd7d1adb6/artifacts


Registered model 'RandomForest' already exists. Creating a new version of this model...
2023/09/19 09:56:02 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForest, version 3
Created version '3' of model 'RandomForest'.


In [18]:
# Adding or Updating an MLflow Model Descriptions

client.update_model_version(
    name="RandomForest",
    version=3,
    description="This a model version 3 description added with update_model_version() method",
)

# Note: if you got an error, make sure you run the previous cell 3 times and save 3 versions of the model! 

<ModelVersion: aliases=[], creation_timestamp=1695110162888, current_stage='None', description='This a model version 3 description added with update_model_version() method', last_updated_timestamp=1695110162946, name='RandomForest', run_id='1c05fa31f83440829d746e7cd7d1adb6', run_link='', source='./mlruns/0/1c05fa31f83440829d746e7cd7d1adb6/artifacts/RandomForest', status='READY', status_message='', tags={}, user_id='', version='3'>

## Discover models and their stages

In [19]:
from pprint import pprint

# Listing and Searching MLflow Models

for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

{   'aliases': {},
    'creation_timestamp': 1695110137966,
    'description': '',
    'last_updated_timestamp': 1695110162888,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1695110162888, current_stage='None', description='This a model version 3 description added with update_model_version() method', last_updated_timestamp=1695110162946, name='RandomForest', run_id='1c05fa31f83440829d746e7cd7d1adb6', run_link='', source='./mlruns/0/1c05fa31f83440829d746e7cd7d1adb6/artifacts/RandomForest', status='READY', status_message='', tags={}, user_id='', version='3'>],
    'name': 'RandomForest',
    'tags': {}}


In [20]:
# Search for a specific model name and list its version details 

for mv in client.search_model_versions("name='RandomForest'"):
    pprint(dict(mv), indent=4)

{   'aliases': [],
    'creation_timestamp': 1695110162888,
    'current_stage': 'None',
    'description': 'This a model version 3 description added with '
                   'update_model_version() method',
    'last_updated_timestamp': 1695110162946,
    'name': 'RandomForest',
    'run_id': '1c05fa31f83440829d746e7cd7d1adb6',
    'run_link': '',
    'source': './mlruns/0/1c05fa31f83440829d746e7cd7d1adb6/artifacts/RandomForest',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '3'}
{   'aliases': [],
    'creation_timestamp': 1695110155270,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1695110155270,
    'name': 'RandomForest',
    'run_id': 'c830d17ae8704aef817989c7e19d0a59',
    'run_link': '',
    'source': './mlruns/0/c830d17ae8704aef817989c7e19d0a59/artifacts/RandomForest',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '2'}
{   'aliases': []

## Transitioninig a model stage

In [21]:
# Over the course of the model’s lifecycle, a model evolves—from development to staging to production. 
# You can transition a registered model to one of the stages: Staging, Production or Archived.

client.transition_model_version_stage(
    name="RandomForest", version=3, stage="Production"
)

<ModelVersion: aliases=[], creation_timestamp=1695110162888, current_stage='Production', description='This a model version 3 description added with update_model_version() method', last_updated_timestamp=1695110165997, name='RandomForest', run_id='1c05fa31f83440829d746e7cd7d1adb6', run_link='', source='./mlruns/0/1c05fa31f83440829d746e7cd7d1adb6/artifacts/RandomForest', status='READY', status_message='', tags={}, user_id='', version='3'>

## Download and use models from the registry

In [22]:
model_version_uri = "models:/example-model@Champion"

In [23]:
# Load the model from the model registry and score
model_uri = f"models:/RandomForest/3"
loaded_model = mlflow.sklearn.load_model(model_uri)
loaded_model

RandomForestRegressor(n_estimators=50, random_state=0)

In [24]:
loaded_model.predict(X_test)

array([ 21.02,  41.12, 131.18,  77.94,  63.72, 143.32,  61.34,   8.36,
         3.38,  19.14,   5.68,  25.14,   9.3 ,  76.44,  51.08, 112.36,
        67.02,   4.18,  86.58,  67.12,  69.88,  97.18,  71.2 ,  12.68,
       183.72,  31.32,  41.38,   2.62,   6.4 ,  48.96, 105.06,  24.62,
        63.78,  46.34,  70.1 ,  87.36,  84.  ,  62.9 , 173.82,  28.36,
        74.34,  58.88,  27.82,  71.82,  99.06,  11.22,  93.04,  51.9 ,
        52.98,  36.58,  64.5 ,  41.42,  87.74,  75.66,  35.82,  52.3 ,
         3.  ,   6.38, 144.62,  11.  ,  10.88, 180.54,  37.18,  64.58,
        51.78,  35.76,  32.5 ,   3.82,  34.3 ,  76.18,  13.62,  47.84,
        60.82,  80.08, 105.44,  80.26,  38.76,   3.74,  35.76,  56.92,
        25.62, 157.86,  32.84,  44.14,  40.14,  38.94,  22.94,  16.42,
        47.58,  28.38, 100.46, 100.02,  29.3 ,  38.6 ,   3.3 ,  79.14,
        77.68,  73.74,  47.78, 127.86,  41.58, 150.42,  48.02,  31.02,
        32.7 ,  69.92,  76.62,  36.38,  55.78, 163.6 ,  78.22, 152.16,
      

## Deregistering, Deleting and Archiving models 

In [25]:
# Archive models version 3 from Production into Archived

client = MlflowClient()
client.transition_model_version_stage(
    name="RandomForest", version=2, stage="Archived"
)

<ModelVersion: aliases=[], creation_timestamp=1695110155270, current_stage='Archived', description='', last_updated_timestamp=1695110167985, name='RandomForest', run_id='c830d17ae8704aef817989c7e19d0a59', run_link='', source='./mlruns/0/c830d17ae8704aef817989c7e19d0a59/artifacts/RandomForest', status='READY', status_message='', tags={}, user_id='', version='2'>

In [26]:
# Note: Deleting registered models or model versions is irrevocable, so use it judiciously.

# Delete version 1 of the model
client.delete_model_version(
        name="RandomForest", version=1,
    )